In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from itertools import chain

In [2]:
root_url = 'https://www.slroutes.com/search'

In [ ]:
response = requests.get(root_url)
response.content
soup = BeautifulSoup(response.content, "html.parser")
search_result = soup.find_all("ul",{"class": "pagination"})

In [ ]:
list_of_pagenumber = []
for i in search_result[1].find_all("a"):
    try:
        int(i.text)
        list_of_pagenumber.append(int(i.text))
    except :
        0
max_pagenumber = max(list_of_pagenumber)

In [5]:
get_pagination_route = search_result[0].find_all("li", {"class":"active jrp-pagination-number"})[0].find_all("a")[0].get("href")[:-1]
get_pagination_route

'/en-US/search?pagenumber='

In [ ]:
link_for_pagination = [root_url[:-len("/search")] + get_pagination_route +str(i) for i in range(1,max_pagenumber)]

In [12]:
page_search_result = []
for link in link_for_pagination:
    response1 = requests.get(link)
    response1.content
    soup1 = BeautifulSoup(response1.content, "html.parser")
    _ = soup1.find_all("a", {"class":"primary-text-color job-result-title"})
    page_search_result.append(_)

In [ ]:
# unest a list of list
page_search_result = list(chain.from_iterable(page_search_result))

In [17]:
page_search_result_links = [root_url[:-len("/search")]+i.get("href") for i in page_search_result]
page_search_result_links[:3]

['https://www.slroutes.com/en-US/job/independent-distributors-business-owner/J3R54660BJWS09BW049',
 'https://www.slroutes.com/en-US/job/independent-distributors-business-owner/J3T33G735H5Z2GNG110',
 'https://www.slroutes.com/en-US/job/independent-distributors-business-owner/J3W6G86177HP5NPG77K']

In [18]:
len(page_search_result_links)

350

In [26]:
# Extracting each field
def extract_field(section, label):
    """Extract the value of a field based on the label."""
    label_tag = section.find("strong", string=lambda x: x and label in x)
    if label_tag:
        # Find the next sibling that contains the actual value
        value = label_tag.find_next_sibling(string=True)
        return value.strip() if value else None
    return None
list_data = []
for page in page_search_result_links:
    # print(page)
    response2 = requests.get(page)
    response2.content
    soup2 = BeautifulSoup(response2.content, "html.parser")

    # Extract location, name, phone number, address, ,weekly avg, and sale price

    # Get the job location
    try:
        job_location = soup2.find_all("span", {"class":"jobLocation"})[0].text
    except:
        job_location = None
    # Get the email
    try:
        email = soup2.find_all("li", {"class":"discrete-field-6"})[0].find_all("div", {"class":"secondary-text-color"})[0].text
    except:
        email = None
    try:
        # Get the job description section
        details = soup2.find_all("div", {"id":"jdp-job-description-section"})[0]
        # Extracting the required data
        route_description_section = details.find("div", class_="jdp-job-description-card content-card")
        # Assign values to variables
        type_ = extract_field(route_description_section, "Type:")
        price = extract_field(route_description_section, "Price:")
        weekly_avg = extract_field(route_description_section, "Weekly Avg:")
        contact_name = extract_field(route_description_section, "Contact Name:")
        contact_phone = extract_field(route_description_section, "Contact Phone:")
    except:
        type = None
        price = None
        weekly_avg = None
        contact_name = None
        contact_phone = None

    # Printing the results

    # print(f"Location: {job_location}")
    # print(f"Email: {email}")
    # print(f"Type: {type_}")
    # print(f"Price: {price}")
    # print(f"Weekly Avg: {weekly_avg}")
    # print(f"Contact Name: {contact_name}")
    # print(f"Contact Phone: {contact_phone}")

    data_dict = {"Location": job_location,
                 "Name": contact_name,
                 "Phone": contact_phone, 
                #  "Email": email,
                # "Type": type_, 
                "Sale Price": price,
                "Weekly Avg": weekly_avg }
    

    _ = pd.DataFrame([data_dict])
    list_data.append(_)

sample = pd.concat(list_data)    
sample.to_csv("sample.csv", index=False)